In [1]:
from google.colab import userdata
from huggingface_hub import login

login(userdata.get('HF_TOKEN'))

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import load_workbook
from sklearn.metrics import confusion_matrix
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax

In [3]:
# model_name = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
model_name = f"finiteautomata/bertweet-base-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
# model.save_pretrained(f"/content/drive/MyDrive/Uni stuff/Evidence-based/{model_name}")

tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [10]:
def compute_sentiment(encoded_input):
    try:
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        top_score = np.max(scores)
        sentiment = config.id2label[np.argmax(scores)]
        return sentiment
    except Exception as e:
        print(f"Error {e} processing encoded input: {encoded_input}")
        return None

def handle_long_text(text):
    max_length = 500
    encoded_input = tokenizer(text, return_tensors='pt')
    input_ids = encoded_input['input_ids'][0]

    if len(input_ids) > max_length:
        parts = [input_ids[i:i + max_length] for i in range(0, len(input_ids), max_length)]
        sentiments = []
        for part in parts:
            part_encoded_input = {'input_ids': part.unsqueeze(0)}
            sentiment = compute_sentiment(part_encoded_input)
            sentiments.append(sentiment)

        sentiment = max(set(sentiments), key=sentiments.count)
        return sentiment
    else:
        return compute_sentiment(encoded_input)

In [11]:
GSHEET_KEY="1YGGro_pvgAgFIB1yx42VShYsTe2zqBimzwubUeofoXY"
sheet_name = "test"
sheet_url=f"https://docs.google.com/spreadsheet/ccc?key={GSHEET_KEY}&output=xlsx"
df = pd.read_excel(sheet_url, sheet_name=sheet_name)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   project           105 non-null    object 
 1   issue_number      105 non-null    int64  
 2   issue_type        105 non-null    object 
 3   text              104 non-null    object 
 4   cleaned_text      104 non-null    object 
 5   human_annotation  105 non-null    object 
 6   chatgpt_text      105 non-null    object 
 7   bert_text         105 non-null    object 
 8   bert_clean        0 non-null      float64
 9   roberta_text      105 non-null    object 
 10  roberta_clean     0 non-null      float64
 11  classification    105 non-null    object 
 12  indicator         105 non-null    object 
dtypes: float64(2), int64(1), object(10)
memory usage: 10.8+ KB


In [12]:
# df['cleaned_text'] = df['cleaned_text'].fillna('None')
# df['roberta_clean'] = df['cleaned_text'].apply(lambda x: pd.Series(handle_long_text(x)))
# df.info()

df['cleaned_text'] = df['cleaned_text'].fillna('None')
df['bert_clean'] = df['cleaned_text'].apply(lambda x: pd.Series(handle_long_text(x)))
df['bert_clean'] = df['bert_clean'].replace({'NEU': 'neutral', 'POS': 'positive', 'NEG': 'negative'})
df.info()

Error index out of range in self processing encoded input: {'input_ids': tensor([[    0, 60566,   856,  7724, 39359,  6770, 54936, 19575,  1646, 19482,
         51544,  9255,  4697,  4697,  1297,  6770, 54936,  2453,   148, 37328,
         10919,  1499,   653,  8949,  5542, 52307,  7724,  4697, 16422,    49,
          8069,   157, 11898,   127,  6770, 54936,   164,   792,  4697,  7953,
          6770, 54936,   143,   507,   372,   813,  2179,   535,  3499, 37328,
          1499,   291, 14317,   483,  1499,  1304,   255, 10919, 60566, 28693,
           635,  8949, 31635,  4285, 60566, 28693,   635, 31635,  7582, 21316,
          2734,  5772, 25681,  1559,  7743, 18378,  2509,   715, 13455, 21316,
         60566,   856,  7724, 18070,   148, 16107,  4697, 36290,   154,  2619,
           861,   157,   785, 16236, 52307,  7724,  8390, 18042,  8949,   755,
          8949, 31635, 17254, 27602, 42195, 31635, 23025, 60566,   856,  7724,
         27699,  8949,  8199,   156,  9512,   148,  7743, 

In [ ]:
df.to_excel(sheet_url, sheet_name=sheet_name, engine='openpyxl')

In [13]:
df.head()

,project,issue_number,issue_type,text,cleaned_text,human_annotation,chatgpt_text,bert_text,bert_clean,roberta_text,roberta_clean,classification,indicator
0,camel,10009,comment_0,If you use then it works,use work,neutral,positive,neutral,neutral,positive,NaN,non_debt,-
1,camel,10009,description,If you use spring and then refer to an endpoin...,use spring refer endpoint route get spring error,neutral,neutral,neutral,neutral,neutral,NaN,non_debt,-
2,camel,10009,summary,Using <to> with id and ref fails,using id ref fails,negative,negative,negative,negative,negative,NaN,non_debt,-
3,camel,10022,comment_0,There is a basic health indicator now.,basic health indicator,neutral,positive,neutral,neutral,neutral,NaN,non_debt,-
4,camel,10022,comment_1,We need the health check API in camel-core for...,need health check api camelcore fine grained s...,neutral,negative,neutral,neutral,neutral,NaN,non_debt,-


In [14]:
# df['agreement'] = df['roberta_text'] == df['roberta_clean']

# agreement_count = df['agreement'].sum()
# disagreement_count = (~df['agreement']).sum()

# agreement_table = pd.DataFrame({
#     'Agreement': [agreement_count],
#     'Disagreement': [disagreement_count]
# })

# agreement_table

df['agreement'] = df['bert_text'] == df['bert_clean']

agreement_count = df['agreement'].sum()
disagreement_count = (~df['agreement']).sum()

agreement_table = pd.DataFrame({
    'Agreement': [agreement_count],
    'Disagreement': [disagreement_count]
})

agreement_table

,Agreement,Disagreement
0,101,4
